In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

#### data_type (유형별 발매 현황)
- 발매채널별 승차권 발매 현황 : 'channel_sales'
- SR 운영역 승차권 발매현황 : 'srt_station_sales'
- 연도별 SR 승차권 발매현황 : 'yearly_sales'
- 특실/일반실 승차인원 : 'room_class_passengers'

In [2]:
# 총 데이터 수 구하는 함수
def getTotalCnt(data_type, PK):
    url = 'https://apis.data.go.kr/B553912/tk_sales/v1/%s?serviceKey=%s&page=1&perPage=10&returnType=JSON&cond[RUN_YMD::LT]=20220101&cond[RUN_YMD::LTE]=20220101&cond[RUN_YMD::GT]=20220101&cond[RUN_YMD::GTE]=20220101'%(data_type, PK)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['totalCount'])
    return cnt

### getAPIconts
---
> __객실 등급별__
> - RUN_YM -> URL의 날짜 설정이 YMD가 아님

> __연도별__
> - 조회 사이트에서 API 링크를 구함(특수기호가 디코드되어 있음)

> __기타__
> - YMD 형식


In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def getAPIconts(data_type, PK, page):
    
    # 최근 일자 구하기(전날)
    yesterday = datetime.today() - timedelta(1) # 가장 최신 일자 (어제)
    yesterday = yesterday.strftime("%Y%m%d")
    
    # 유형별 URL 조합하기
    if data_type == 'room_class_passengers':
        url = 'https://apis.data.go.kr/B553912/tk_sales/v1/%s?serviceKey=%s&page=%s&perPage=10000&returnType=JSON&cond[RUN_YM::LT]=%s&cond[RUN_YM::LTE]=%s&cond[RUN_YM::GT]=20220801&cond[RUN_YM::GTE]=20220801'%(data_type, key,str(page), yesterday, yesterday) 
    elif data_type == 'yearly_sales':
        url = 'https://apis.data.go.kr/B553912/tk_sales/v1/yearly_sales?page=1&perPage=10000&cond%5BRUN_YR%3A%3AGTE%5D=2022&cond%5BRUN_YR%3A%3ALTE%5D=2023&serviceKey=B7y6mnXwYiqT8J24zK9IAyF7EGDjLktqZMwR%2F0h8yg6Tq6Scw%2Fj5naZkQIInZ6b2iM%2B6Ndvk%2FFrG4qoTK3gK1Q%3D%3D' 
    else:
        url = 'https://apis.data.go.kr/B553912/tk_sales/v1/%s?serviceKey=%s&page=%s&perPage=10000&returnType=JSON&cond[RUN_YMD::LT]=%s&cond[RUN_YMD::LTE]=%s&cond[RUN_YMD::GT]=20220801&cond[RUN_YMD::GTE]=20220801'%(data_type, key,str(page),yesterday, yesterday)
    
    # API 호출하기
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['data']
    
    return data

### API 가져오기
---
> __발매 채널별 / SR 운영역 별 - 10000개 이상 데이터__ 
>    - 위에서 구한 총 데이터 수량으로 페이지 수 구함

> __연도별 / 객실 등급별__
> - 페이지 1로 고정

> __공통__
>    - 한 페이지 당  10000개 씩 로드
>    - 최근 데이터 : 기준일로부터 이틀 전

In [4]:
# 발매 채널별
channel_sale = []
cnt = getTotalCnt('channel_sales', key)
page = int(cnt/10000) + 1
for i in range(1,page+1):
    js = getAPIconts('channel_sales', key,i)
    for item in js:
        channel_sale.append(item)

# SR 운영역별        
sr_station_sale = []
cnt = getTotalCnt('srt_station_sales', key)
page = int(cnt/10000) + 1
for i in range(1, page+1):
    js = getAPIconts('srt_station_sales', key,i)
    for item in js:
        sr_station_sale.append(item)

# 연도별
yearly_sale = getAPIconts('yearly_sales', key, 1)

# 좌석 등급별
roomclass_sale = getAPIconts('room_class_passengers', key,1)

### 데이터 프레임 변환

In [5]:
channel_sale_df = pd.json_normalize(channel_sale)
sr_station_sale_df = pd.json_normalize(sr_station_sale)
yearly_sale_df = pd.json_normalize(yearly_sale)
roomclass_sale_df = pd.json_normalize(roomclass_sale)


### (1) 발매 채널별 발매 현황

In [6]:
channel_sale_df.columns = ['인쇄완료수량','환불수량','노선명','운행일자','발매수량','발매창구종류코드','발매창구종류명','발매일자']
channel_sale_df

,인쇄완료수량,환불수량,노선명,운행일자,발매수량,발매창구종류코드,발매창구종류명,발매일자
0,97,30,경부선,20220802,129,0011,역,20220801
1,4,0,경부선,20220802,4,0022,TMO(군장병 철도수송지원반),20220801
2,61,19,경부선,20220802,80,0027,철도공사 자회사 위탁운영역,20220801
3,8,1,경부선,20220802,9,0036,현금ATM,20220801
4,10,5,경부선,20220802,15,0037,Tablet(자동발매기),20220801
...,...,...,...,...,...,...,...,...
39291,5,2,호남선,20230107,7,0027,철도공사 자회사 위탁운영역,20230107
39292,129,5,호남선,20230107,134,0036,현금ATM,20230107
39293,84,5,호남선,20230107,89,0037,Tablet(자동발매기),20230107
39294,4846,2066,호남선,20230107,6961,0077,스마트폰,20230107


### (2) SR 운영역 승차권 발매현황

In [7]:
sr_station_sale_df.columns = ['인쇄완료수량','환불수량','노선명','운행일자','발매수량','발매역코드','발매역명','발매일자']
sr_station_sale_df

,인쇄완료수량,환불수량,노선명,운행일자,발매수량,발매역코드,발매역명,발매일자
0,35,18,경부선,20220802,53,0551,수서,20220801
1,5,1,경부선,20220802,6,0552,동탄,20220801
2,23,3,경부선,20220802,26,0553,지제,20220801
3,28,15,호남선,20220802,43,0551,수서,20220801
4,2,3,호남선,20220802,5,0552,동탄,20220801
...,...,...,...,...,...,...,...,...
14180,148,18,경부선,20230107,168,0552,동탄,20230107
14181,147,17,경부선,20230107,165,0553,지제,20230107
14182,261,35,호남선,20230107,298,0551,수서,20230107
14183,81,2,호남선,20230107,83,0552,동탄,20230107


### (3) 연도별 SR 승차권 발매현황

In [8]:
yearly_sale_df.columns = ['인쇄완료수량','환불수량','노선명','운행연도','발매수량']
yearly_sale_df

,인쇄완료수량,환불수량,노선명,운행연도,발매수량
0,7242109,4964883,경부선,2022,12301288
1,2798540,1994762,호남선,2022,4841702


### (4) 특실/일반실 승차인원

In [9]:
roomclass_sale_df.columns = ['객실등급코드','객실등급명','노선명','운행연월','승차인원수']
roomclass_sale_df

,객실등급코드,객실등급명,노선명,운행연월,승차인원수
0,1,일반실,경부선,202209,1328839
1,2,특실,경부선,202209,103303
2,1,일반실,호남선,202209,509760
3,2,특실,호남선,202209,41270
4,1,일반실,경부선,202210,1471857
5,2,특실,경부선,202210,111154
6,1,일반실,호남선,202210,559321
7,2,특실,호남선,202210,44465
8,1,일반실,경부선,202211,1397660
9,2,특실,경부선,202211,105675
